In [3]:
%load_ext autoreload
%autoreload 2
from main import *
import random, string
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
from metrics import compute_topk_contains_target

## Setup Config

In [4]:
domain = 'pollini'
config_path = "config/" + domain
config = Config()
config.load_from_file(config_path)

data = "data/"+domain
pages = torch.load(data+"/pages.pt")
actions = torch.load(data+"/actions.pt")
node_ids= torch.load(data+"/n2f.pt")
nodes2labels = {key: pages[page]+"_"+actions[action] for key, (page,action) in node_ids.items()}
logging = SummaryWriter()

In [5]:
nodes2labels

{0: 'Cart_Click',
 1: 'Product_scroll',
 2: 'Cart_placeorder',
 3: 'Product_addtocart',
 4: 'ContactUs_scroll',
 5: 'Home_checkout',
 6: 'info_Click',
 7: 'info_placeorder',
 8: 'Category_Click',
 9: 'Category_placeorder',
 10: 'placeorder_scroll',
 11: 'UserAccount_checkout',
 12: 'Cart_checkout',
 13: 'Product_Click',
 14: 'Product_placeorder',
 15: 'ContactUs_Click',
 16: 'info_checkout',
 17: 'Category_checkout',
 18: 'Home_scroll',
 19: 'placeorder_Click',
 20: 'UserAccount_scroll',
 21: 'placeorder_placeorder',
 22: 'Product_checkout',
 23: 'Cart_addtocart',
 24: 'ContactUs_checkout',
 25: 'Cart_scroll',
 26: 'Home_Click',
 27: 'info_scroll',
 28: 'Category_addtocart',
 29: 'info_addtocart',
 30: 'Category_scroll',
 31: 'placeorder_checkout',
 32: 'UserAccount_Click'}

## Setup Model

In [3]:
def get_balance_acc(count, acc):
    temp = count.vec_sum/acc.vec_sum
    return torch.isfinite(temp).sum(), torch.isnan(temp).sum()

def setup_model(config):
    TRAJ_MAX_LEN=99
    graph, trajectories, pairwise_node_features, _ = load_data(config)

    model = create_model(graph, pairwise_node_features, config)
    model = model.to(config.device)
    train_trajectories, valid_trajectories, test_trajectories = trajectories
    use_validation_set = len(valid_trajectories) > 0

    graph = graph.to(config.device)
    given_as_target, siblings_nodes = None, None

    if config.enable_checkpointing:
            chkpt_dir = os.path.join(
                config.workspace, config.checkpoint_directory, config.name)
            os.makedirs(chkpt_dir, exist_ok=True)

    optimizer = create_optimizer(model.parameters(), config)

    if config.restore_from_checkpoint:
        filename = input("Checkpoint file: ")
        checkpoint_data = torch.load(filename)
        model.load_state_dict(checkpoint_data["model_state_dict"])
        optimizer.load_state_dict(checkpoint_data["optimizer_state_dict"])
        print("Loaded parameters from checkpoint")

    if config.compute_baseline:
        display_baseline(config, graph, train_trajectories,
                         test_trajectories, create_evaluator())

    graph = graph.add_self_loops(degree_zero_only=config.self_loop_deadend_only, edge_value=config.self_loop_weight)
    return model, graph, pairwise_node_features, optimizer, train_trajectories, valid_trajectories, test_trajectories, given_as_target, siblings_nodes,use_validation_set

In [64]:
def gen_metrics(model1, model2, graph, n_obs1, n_obs2, trajectories, config, pairwise_features, alpha, k=5):
    top1_max = []
    top1_mean = []
    m1 = []
    m2 = []
    model1.eval()
    model2.eval()
    with torch.no_grad():
        for trajectory_idx in range(len(trajectories)):
            observations = trajectories[trajectory_idx]

            number_steps = None
            if config.rw_edge_weight_see_number_step or config.rw_expected_steps:
                if config.use_shortest_path_distance:
                    number_steps = (
                        trajectories.leg_shortest_lengths(
                            trajectory_idx).float() * 1.1
                    ).long()
                else:
                    number_steps = trajectories.leg_lengths(trajectory_idx)

            ## Model 1
            observed, starts, targets = generate_masks(
                trajectory_length=observations.shape[0],
                number_observations=n_obs1,
                predict=config.target_prediction,
                with_interpolation=config.with_interpolation,
                device=config.device,
            )

            diffusion_graph = (
                graph if not config.diffusion_self_loops else graph.add_self_loops()
            )
            
            predictions1, _, rw_weights = model1(
            observations,
            graph,
            diffusion_graph,
            observed=observed,
            starts=starts,
            targets=targets,
            pairwise_node_features=pairwise_features,
            number_steps=number_steps)
            len_targ1 = len(targets)
            
            ## Model 2
            observed, starts, targets = generate_masks(
                trajectory_length=observations.shape[0],
                number_observations=n_obs2,
                predict=config.target_prediction,
                with_interpolation=config.with_interpolation,
                device=config.device,
            )
            len_targ2 = len(targets)
            predictions2, _, rw_weights = model2(
            observations,
            graph,
            diffusion_graph,
            observed=observed,
            starts=starts,
            targets=targets,
            pairwise_node_features=pairwise_features,
            number_steps=number_steps)

            min_len = min(len_targ1, len_targ2)

            ## Combine Models
            target_distributions = observations[-min_len:, :]
            predictions1 = predictions1[-min_len:,:]
            predictions2 = predictions2[-min_len:,:]

            #print(target_distributions.shape, predictions1.shape, predictions2.shape)
            m1.append(compute_topk_contains_target(target_distributions, predictions1, k=k))
            m2.append(compute_topk_contains_target(target_distributions, predictions2, k=k))

            predictions = torch.max(torch.stack((predictions1, predictions2)),dim=0).values
            top1_max.append(compute_topk_contains_target(target_distributions, predictions, k=k))

            predictions = torch.add(predictions1*alpha, (1-alpha)*predictions2)
            top1_mean.append(compute_topk_contains_target(target_distributions, predictions, k=k))
            #print(predictions1.shape, predictions2.shape,compute_topk_contains_target(target_distributions, predictions, k=1) )
    
    top_max = torch.cat(top1_max)
    top_mean = torch.cat(top1_mean)
    m1_mean = torch.cat(m1)
    m2_mean = torch.cat(m2)
    
    #print(top_max, top_mean)
    return top_max.float().mean(), top_mean.float().mean(), m1_mean.float().mean(), m2_mean.float().mean()

## Train

In [5]:
def train(config, logging):
    model, graph, pairwise_node_features, optimizer, train_trajectories, valid_trajectories, test_trajectories, given_as_target, siblings_nodes,use_validation_set = setup_model(config)
    for epoch in range(config.number_epoch):
        print("EPOCH: " , epoch)
        model.train()

        print_cum_loss = 0.0
        print_num_preds = 0
        print_time = time.time()
        print_every = len(
            train_trajectories) // config.batch_size // config.print_per_epoch

        trajectories_shuffle_indices = np.arange(len(train_trajectories))
        if config.shuffle_samples:
            np.random.shuffle(trajectories_shuffle_indices)

        for iteration, batch_start in enumerate(range(0, len(trajectories_shuffle_indices) - 
                                                      config.batch_size + 1, config.batch_size)):

                optimizer.zero_grad()
                loss = torch.tensor(0.0, device=config.device)

                for i in range(batch_start, batch_start + config.batch_size):
                    trajectory_idx = trajectories_shuffle_indices[i]
                    observations = train_trajectories[trajectory_idx]
                    length = train_trajectories.lengths[trajectory_idx]

                    number_steps = None
                    if config.rw_edge_weight_see_number_step or config.rw_expected_steps:
                        if config.use_shortest_path_distance:
                            number_steps = (
                                train_trajectories.leg_shortest_lengths(
                                    trajectory_idx).float() * 1.1
                            ).long()
                        else:
                            number_steps = train_trajectories.leg_lengths(
                                trajectory_idx)

                    observed, starts, targets = generate_masks(
                        trajectory_length=observations.shape[0],
                        number_observations=config.number_observations,
                        predict=config.target_prediction,
                        with_interpolation=config.with_interpolation,
                        device=config.device,
                    )

                    diffusion_graph = graph if not config.diffusion_self_loops else graph.add_self_loops()

                    predictions, potentials, rw_weights = model(
                        observations,
                        graph,
                        diffusion_graph,
                        observed=observed,
                        starts=starts,
                        targets=targets,
                        pairwise_node_features=pairwise_node_features,
                        number_steps=number_steps,
                    )

                    print_num_preds += starts.shape[0]

                    l = (compute_loss(config.loss,train_trajectories, observations, predictions,starts,targets,rw_weights, trajectory_idx,)
                        / starts.shape[0])
                    loss += l

                loss /= config.batch_size
                print_cum_loss += loss.item()
                loss.backward()
                optimizer.step()

                if (iteration + 1) % print_every == 0:
                    print_loss = print_cum_loss / print_every
                    print_loss /= print_num_preds
                    pred_per_second = 1.0 * print_num_preds / \
                        (time.time() - print_time)

                    print_cum_loss = 0.0
                    print_num_preds = 0
                    print_time = time.time()

                    progress_percent = int(
                        100.0 * ((iteration + 1) // print_every) /
                        config.print_per_epoch
                    )

                    #print(
                    #    f"Progress {progress_percent}% | iter {iteration} | {pred_per_second:.1f} pred/s | loss {config.loss} {print_loss}"
                    #)
        # VALID and TEST metrics computation
        def create_evaluator():
            return Evaluator(
                graph.n_node,
                given_as_target=given_as_target,
                siblings_nodes=siblings_nodes,
                config=config,
            )
        test_evaluator, node_acc_dist = evaluate(model,graph,test_trajectories,pairwise_node_features,create_evaluator,dataset="TEST",)

        to_fin, to_nan = get_balance_acc(node_acc_dist['node_to_count'],node_acc_dist['node_to_acc5'])
        from_fin, from_nan = get_balance_acc(node_acc_dist['node_from_count'],node_acc_dist['node_from_acc5'])
        zeros5_to = torch.count_nonzero(node_acc_dist['node_to_acc5'].vec_sum).sum()
        zeros5_from = torch.count_nonzero(node_acc_dist["node_from_acc5"].vec_sum).sum()
        zeros1_to = torch.count_nonzero(node_acc_dist['node_to_acc1'].vec_sum).sum()
        zeros1_from = torch.count_nonzero(node_acc_dist["node_from_acc1"].vec_sum).sum()
        
        ## Checkpointing
        if config.enable_checkpointing and epoch % config.chechpoint_every_num_epoch == 0:
            # print("Checkpointing...")
            directory = os.path.join(
                config.workspace, config.checkpoint_directory, config.name)
            chkpt_file = os.path.join(directory, f"{epoch:04d}.pt")
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                },
                chkpt_file,
            )
            config_file = os.path.join(directory, "config")
            config.save_to_file(config_file)

            metrics_file = os.path.join(directory, f"{epoch:04d}.txt")
            with open(metrics_file, "w") as f:
                f.write(test_evaluator.to_string())
        ## log hyperparameters
        logging.add_hparams({'batch_size': config.batch_size, 'lr': config.lr,
                            'self_loop_weight': config.self_loop_weight,
                            'loss':config.loss,
                            'target': config.target_prediction, "n_obs":config.number_observations},
                            {"top1":test_evaluator.metrics['precision_top1'].mean(), 
                            "top5":test_evaluator.metrics['precision_top5'].mean(),
                            })
    
    accuracy = node_acc_dist['node_to_acc5'].vec_sum.cpu()/node_acc_dist['node_to_count'].vec_sum.cpu()
    nodes_not_predicted = np.arange(len(accuracy))[accuracy<0.2]
    print("-----------------------------------------------------------")
    print("Diversity:", [nodes2labels[i] for i in nodes_not_predicted])
    print("-----------------------------------------------------------")
    #return zeros1_to, zeros5_to, zeros1_from, zeros5_from, node_acc_dist, model
    return model, graph, test_trajectories, pairwise_node_features

## Grid search

In [68]:
## Hyperparameters -- defaults
logging = SummaryWriter()
config.target_prediction = "next"
config.lr = 0.002
config.batch_size = 24
config.self_loop_weight = 0.01
config.loss = 'nll_loss'
config.diffusion_k_hops = 1
config.number_epoch = 3

## Graph diffusion params
rw_expected_steps = True
rw_edge_weight_see_number_step = True
with_interpolation = True
initial_edge_transformer = True
use_shortest_path_distance = False
double_way_diffusion = False
rw_non_backtracking = False
diffusion_self_loops = True

bools = [True, False]

## Define grid
lrs = [0.005]
bss = [24]
loop_weights = [0.01]
loss = ["nll_loss"]
logs = []
#for num in num_obss:
#    for hops in [2]:
#        config.diffusion_k_hops = hops
#        config.number_observations = num
#        random_name = ''.join(random.choice(string.ascii_lowercase) for i in range(6))
#        zeros1_to, zeros5_to, zeros1_from, zeros5_from, node_acc_dist = train(config, logging)

## Combine two models

In [70]:
n_obs1 = 2
n_obs2 = 1
config.number_observations = n_obs1
model1, graph, test_trajectories, pairwise_node_features = train(config, logging)

config.number_observations = n_obs2
model2, graph, _, _ = train(config, logging)
random_name = ''.join(random.choice(string.ascii_lowercase) for i in range(6))
for alpha in [0.5]:
    top_max, top_mean, m1, m2 = gen_metrics(model1, model2, graph, n_obs1, n_obs2, test_trajectories, config, pairwise_node_features, alpha)
    print(top_max.item(), top_mean.item(), m1.item(), m2.item())

EPOCH:  0
{'precision_top1': 0.43887255749610926, 'precision_top2': 0.635656233788691, 'precision_top3': 0.7065536918554384, 'precision_top5': 0.8077122600726266}
EPOCH:  1
{'precision_top1': 0.4625626837281688, 'precision_top2': 0.6764655023344285, 'precision_top3': 0.7499567698426423, 'precision_top5': 0.8689261628912329}
EPOCH:  2
{'precision_top1': 0.45737506484523605, 'precision_top2': 0.6714508040809268, 'precision_top3': 0.7527234999135397, 'precision_top5': 0.8734220992564413}
-----------------------------------------------------------
Diversity: ['ContactUs_scroll', 'Product_placeorder', 'Category_checkout', 'Cart_scroll', 'Category_addtocart']
-----------------------------------------------------------
EPOCH:  0
{'precision_top1': 0.4564868336544637, 'precision_top2': 0.6204238921001927, 'precision_top3': 0.724791265253693, 'precision_top5': 0.8142260757867694}
EPOCH:  1
{'precision_top1': 0.4824983943481053, 'precision_top2': 0.6881824020552344, 'precision_top3': 0.764611432

In [60]:
generate_masks(11, 10)

(tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]), tensor([9]), tensor([10]))

In [59]:
generate_masks(11, 2)

(tensor([[0, 1],
         [1, 2],
         [2, 3],
         [3, 4],
         [4, 5],
         [5, 6],
         [6, 7],
         [7, 8],
         [8, 9]]),
 tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10]))

## Plots

In [ ]:
plt.figure(figsize=(14, 6))
plt.subplot(1,2,1)
plt.title("Prediction")
plt.bar(np.arange(node_acc_dist['node_to_acc5'].vec_sum.shape[0]), node_acc_dist['node_to_acc5'].vec_sum.cpu())
plt.subplot(1,2,2)
plt.title("Actual")
plt.bar(np.arange(node_acc_dist['node_to_acc5'].vec_sum.shape[0]), node_acc_dist['node_to_count'].vec_sum.cpu())
#plt.ylim([0, 200])

In [ ]:
preds = node_acc_dist['node_to_acc5'].vec_sum.cpu()
trues = node_acc_dist['node_to_count'].vec_sum.cpu()

accuracy = preds/trues
nodes_not_predicted = np.arange(len(accuracy))[accuracy<0.2]
[nodes2labels[i] for i in nodes_not_predicted]